In [ ]:
import torch
from data import FloorDataset
from torch.utils.data import DataLoader
from utils import divide_set, input_to_image, plot_imgs_align, masks_to_colorimg

In [ ]:
floor_dataset  = FloorDataset(img_dir = '/home/reza_voxelsafety_com/experiments/segmentation/data/floor_segmentation/train/images/', mask_dir = '/home/reza_voxelsafety_com/experiments/segmentation/data/floor_segmentation/train/annotations/', num_class = 2)
print("Number of images in the dataset {}".format(len(floor_dataset)))
train, val = divide_set(floor_dataset)
print(len(val))

In [ ]:
def get_subset(indices, start, end):
    return indices[start : start + end]


train_ratio, test_ratio = 0.8, 0.2  # rest will go for test
train_count = int(len(floor_dataset) * train_ratio)
test_count = int(len(floor_dataset) * train_ratio)

indices = torch.randperm(len(floor_dataset))

train_indices = get_subset(indices, 0, train_count)
test_indices = get_subset(indices, train_count, test_count)
print(train_indices.shape)
print(test_indices.shape)

In [ ]:
bas_dir = '/home/reza_voxelsafety_com/experiments/segmentation/data/floor_segmentation'
from PIL import Image
import os 
train_names = [floor_dataset._names[i] for i in train_indices.tolist()]
test_names = [floor_dataset._names[i] for i in test_indices.tolist()]
for name in train_names:
    img = Image.open(os.path.join(bas_dir, 'images', name + '.jpg'))
    mask = Image.open(os.path.join(bas_dir, 'annotations', name + '.png'))
    img.save(os.path.join(bas_dir, 'train' ,'images', name + '.jpg'))
    mask.save(os.path.join(bas_dir, 'train' , 'annotations', name + '.png'))
for name in test_names:
    img = Image.open(os.path.join(bas_dir, 'images', name + '.jpg'))
    mask = Image.open(os.path.join(bas_dir, 'annotations', name + '.png'))
    img.save(os.path.join(bas_dir, 'test' ,'images', name + '.jpg'))
    mask.save(os.path.join(bas_dir, 'test' , 'annotations', name + '.png'))


In [ ]:
floor_dataset  = FloorDataset(img_dir = '/home/reza_voxelsafety_com/experiments/segmentation/data/floor_segmentation/test/images/', mask_dir = '/home/reza_voxelsafety_com/experiments/segmentation/data/floor_segmentation/test/annotations/', num_class = 2)
print("Number of images in the dataset {}".format(len(floor_dataset)))

In [ ]:
batch_size = 4
dataloaders = {
    'train': DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=8),
    'val': DataLoader(val, batch_size=batch_size, shuffle=True, num_workers=8),
}
input_img, mask = next(iter(dataloaders['train'])) # getting a batch of the dataset for demonstarion
print(input_img.shape)
input_images_rgb = [input_to_image(x) for x in input_img.cpu()][:4]  
target_masks_rgb = [masks_to_colorimg(x) for x in mask.cpu().numpy()][:4]
plot_imgs_align([input_images_rgb, target_masks_rgb], None)

In [ ]:
import torch
import torch.nn as nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # setting the device for the process
print(device)

In [ ]:
from collections import defaultdict
import torch.nn.functional as F
from loss import calc_loss

def print_metrics(metrics, epoch_samples, phase):    
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))
        
    print("{}: {}".format(phase, ", ".join(outputs)))    



def train_model(model, optimizer, data_loaders, scheduler, num_epochs = 25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    model.train() # set the model to train mode
    
    for epoch in range(num_epochs):
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        since = time.time()
        for phase in ['train', 'val']:
            if phase == 'train':
                
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])  
                model.train()
                
            else:
                model.eval()   
            metrics = defaultdict(float)
            epoch_samples = 0
            
            for inputs, labels in data_loaders[phase]:
               
                
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs.type(torch.float32))
                    loss = calc_loss(outputs, labels, metrics)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
            
                epoch_samples += inputs.size(0)
            
            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples
            
            if phase == 'val' and epoch_loss < best_loss:
                print("saving best model")
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
            else:
                scheduler.step()
            
        

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
from collections import defaultdict
import torch.nn.functional as F
from loss import calc_loss
from tqdm import tqdm 

def print_metrics(metrics, epoch_samples, phase):    
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))
        
    print("{}: {}".format(phase, ", ".join(outputs)))    



def train_model(model, optimizer, data_loaders, scheduler, num_epochs = 25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    model.train() # set the model to train mode
    
    for epoch in range(num_epochs):
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        since = time.time()
        for phase in ['train', 'val']:
            if phase == 'train':
                
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])  
                model.train()
                
            else:
                model.eval()   
            metrics = defaultdict(float)
            epoch_samples = 0
            with tqdm(data_loaders[phase], unit="batch") as tepoch:
                for inputs, labels in tepoch:
                
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs.type(torch.float32))
                        loss = calc_loss(outputs, labels, metrics)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()
                
                    epoch_samples += inputs.size(0)
                    tepoch.set_postfix(loss=loss.item())
            
            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples
            
            if phase == 'val' and epoch_loss < best_loss:
                print("saving best model")
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
            else:
                scheduler.step()
            
        

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
import torch
import torch.optim as optim
import time
import copy
from collections import defaultdict
from torch.optim import lr_scheduler
from fastai.vision.learner import create_unet_model
# setting up the model structure using smp library (listed in the report)
from fastai.vision.models import resnet34
model = create_unet_model(resnet34, 2, (320, 480), True, n_in=3)


model = model.to(device) # passing the model to the device (cpu/gpu)
learning_rate = 1e-3

optimizer_ft = optim.Adam(model.parameters(), lr = 1e-3)

lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 5,
                    gamma = 1e-1)

model = train_model(model, optimizer_ft, dataloaders, lr_scheduler ,num_epochs = 10)

In [ ]:
save_path = './data/floor__best_model.pth' # model parameters save name
torch.save(model.state_dict(), save_path) # save model parameters 

In [ ]:
from iou import IoU
# import numpy as np
# save_path = './data/floor__best_model.pth' # model parameters save name

# per_class_mean_accuracy = [] # per class IoU
# mean_Accuracy = [] # average IoU
# iter = 0
# model = smp.Unet(
#     encoder_name = "resnet34",        
#     encoder_weights = 'imagenet',     
#     in_channels = 3,                 
#     classes = 2,                      
# )
# model.load_state_dict(torch.load(save_path))
model = model.to(device)
model.eval()  # model is set for the test process
iteration = 0
for inputs, labels in dataloaders['test']:
    inputs = inputs.to(device) # pasing input to the device
    labels = labels.to(device) # passing label to the device
    pred = model(inputs.type(torch.float32)) # passing images to the model and predict

    pred = pred.data.cpu().numpy()  # bring back the prediction from Gpu to cpu 
    if iteration % 100 == 0:

        input_images_rgb = [input_to_image(x) for x in inputs.cpu()] # input rgb images for visualization
        target_masks_rgb = [masks_to_colorimg(x) for x in labels.cpu().numpy()] # mask labels (ground truth) to color image for visualization
        pred_rgb = [masks_to_colorimg(x) for x in pred] # convert the prediction to images for visualization
        
        plot_imgs_align([input_images_rgb, target_masks_rgb, pred_rgb]) # plot images side by side
    labels = labels.data.cpu().numpy()
    pred_label = [x for x in pred]

    pred_label = np.asarray(pred_label)

    metric = IoU(num_classes = 2)
    metric.add(torch.from_numpy(pred_label), torch.from_numpy(labels)) # calculating IoU based on the library we cited 
    per_class_IoU, mean_IoU  = metric.value() 
    per_class_mean_accuracy.append(per_class_IoU) # append the per class IoU

    print("Current Batch: per class IoU: {} and average IoU {}: ".format(per_class_IoU, mean_IoU)) # print the average and per class IoU for each batch of data
    print("-----------------------------")
    mean_Accuracy.append(mean_IoU) # append the average Iou
    iter +=1

In [ ]:
from fastai.data.core import DataLoaders

In [ ]:
dls = DataLoaders.from_dsets(train, val, bs = 8).cuda()

In [ ]:
print(dls)

In [ ]:
b = dls.one_batch()
print(b[1].shape)

In [ ]:
class unet_loss:
    "Dice and Focal combined"
    def __init__(self, bce_weight= 0.5, smooth = 0.1):
        self._weight = bce_weight
        self._smooth = smooth
    def _diceloss(self, pred, target):
        pred = pred.contiguous()
        target = target.contiguous()    

        intersection = (pred * target).sum(dim=2).sum(dim=2)
    
        loss = (1 - ((2. * intersection + self._smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + self._smooth)))
        return loss.mean()
    def __call__(self, pred, target):
        
        bce = F.binary_cross_entropy_with_logits(pred, target)
        pred = torch.sigmoid(pred)
        dice = self._diceloss(pred, target)
    
        loss = bce * self._weight + dice * (1 - self._weight)
        return loss

In [ ]:
import sys
sys.path.append("../..")
import copy

from core.ml.training.dataloaders.registry import get_dataloader
from core.ml.training.loss.registry import get_loss
from core.ml.training.metrics.registry import get_metrics
from core.ml.training.models.registry import get_model_deprecated
from core.perception.inference.transforms.registry import get_transforms

from fastai.vision.all import *
from core.perception.inference.transforms.registry import get_transforms
config_path = '/home/reza_voxelsafety_com/voxel/core/ml/training/configs/floor_segment_training.yaml'
import yaml
with open(config_path, encoding="utf-8") as config_text:
        parsed_config = yaml.safe_load(config_text)
        import copy
learner_init_params = copy.deepcopy(
            parsed_config["learner"]["init_params"]
        )
learner_init_params["metrics"] = [
            get_metrics(name)
            for name in learner_init_params.pop("metrics", [])
        ]

print(learner_init_params)

In [ ]:
print(learner_init_params)

In [ ]:
from fastai.vision.models import resnet34
model = create_unet_model(resnet34, 2, (480, 320), True, n_in=3)
model.to(device)
model.train()
loss_fn = unet_loss(bce_weight= 0.5, smooth = 0.1)
learner = Learner(dls, model, loss_fn)
learner_fit_params = copy.deepcopy(
            parsed_config["learner"]["fit_params"]
        ) 
learner.fit(**learner_fit_params)


In [ ]:
torch.save(learner.model, "/home/reza_voxelsafety_com/experiments/segmentation/data/fast_ai_floorsegmentation.pth") # or save it's state_dict, better option

In [ ]:
from torch.utils.data import DataLoader
print(len(test))
dl_test = DataLoader(test, batch_size=16, shuffle=False, num_workers=8)

In [ ]:
#dl_test = DataLoader(test, batch_size=16, shuffle=False, num_workers=8)
learner.model.eval()  # model is set for the test process
iteration = 0
for inputs, labels in dl_test:
    inputs = inputs.to(device) # pasing input to the device
    labels = labels.to(device) # passing label to the device
    pred = learner.model(inputs.type(torch.float32)) # passing images to the model and predict

    pred = pred.data.cpu().numpy()  # bring back the prediction from Gpu to cpu 
    if iteration % 100 == 0:

        input_images_rgb = [input_to_image(x) for x in inputs.cpu()] # input rgb images for visualization
        target_masks_rgb = [masks_to_colorimg(x) for x in labels.cpu().numpy()] # mask labels (ground truth) to color image for visualization
        pred_rgb = [masks_to_colorimg(x) for x in pred] # convert the prediction to images for visualization
        
        plot_imgs_align([input_images_rgb, target_masks_rgb, pred_rgb]) # plot images side by side
    labels = labels.data.cpu().numpy()
    pred_label = [x for x in pred]

    pred_label = np.asarray(pred_label)

    metric = IoU(num_classes = 2)
    metric.add(torch.from_numpy(pred_label), torch.from_numpy(labels)) # calculating IoU based on the library we cited 
    per_class_IoU, mean_IoU  = metric.value() 
    per_class_mean_accuracy.append(per_class_IoU) # append the per class IoU

    print("Current Batch: per class IoU: {} and average IoU {}: ".format(per_class_IoU, mean_IoU)) # print the average and per class IoU for each batch of data
    print("-----------------------------")
    mean_Accuracy.append(mean_IoU) # append the average Iou
    iter +=1